In [ ]:
from dask.distributed import Client, progress
import dask
import pandas as pd
import fsspec 
import numpy as np
import xarray as xr

In [ ]:
# Setting up clients for DASK and Amazon webservices

try:
    from dask.distributed import get_client
    get_client().close()
except Exception:
    pass

client = Client()  # set up local cluster on your laptop
client

fs = fsspec.filesystem('s3', anon=True)

2025-09-24 05:43:40,662 - distributed.scheduler - WARNING - Worker failed to heartbeat for 8839s; attempting restart: <WorkerState 'tcp://127.0.0.1:52188', name: 1, status: running, memory: 0, processing: 0>
2025-09-24 05:43:40,667 - distributed.scheduler - WARNING - Worker failed to heartbeat for 8839s; attempting restart: <WorkerState 'tcp://127.0.0.1:52189', name: 2, status: running, memory: 0, processing: 0>
2025-09-24 05:43:40,668 - distributed.scheduler - WARNING - Worker failed to heartbeat for 8839s; attempting restart: <WorkerState 'tcp://127.0.0.1:52194', name: 3, status: running, memory: 0, processing: 0>
2025-09-24 05:43:40,669 - distributed.scheduler - WARNING - Worker failed to heartbeat for 8839s; attempting restart: <WorkerState 'tcp://127.0.0.1:52195', name: 0, status: running, memory: 0, processing: 0>
2025-09-24 05:43:41,197 - distributed.nanny - WARNING - Restarting worker
2025-09-24 05:43:41,242 - distributed.nanny - WARNING - Restarting worker
2025-09-24 05:43:41,

In [49]:
# Setting up the base url to retrieve precipitation files. There are two values of precipitation
# in this dataset, large-scale precipitation (lsp) and convective precipitation (cp). To find total precipitation
# you must combine the two. There are also multiple files per month, so I set up the framework for each type
# of precip per month, and then use fs.glob to find all the relevant files.

base_url = 's3://nsf-ncar-era5/e5.oper.fc.sfc.accumu/'
base_url2 = '/e5.oper.fc.sfc.accumu.128_'
base_url3 = '_*.nc'

lsp_tag = '142'
cp_tag = '143'

center_coords = [47.5, -122.3] # latitude/longitude for Seattle, Wa

lat_bounds = [center_coords[0] + 5, center_coords[0] - 5]
lon_bounds = [center_coords[1]- 5 + 360, center_coords[1] + 5 + 360]

# period of interest
pr = pd.date_range(start='1990-01',end='2020-12', freq='MS')

lsp_file_list=[]
cp_file_list=[]
for dt in pr:
    
    year_month = dt.strftime('%Y%m')

    # build glob file names for cp and lsp
    
    # lsp
    single_file=(base_url+year_month+base_url2+lsp_tag+base_url3)
    lsp_file_list.append(single_file)

    # cp
    single_file=(base_url+year_month+base_url2+cp_tag+base_url3)
    cp_file_list.append(single_file)

In [42]:
# using fs.glob to construct the complete file list.
lsp_final_list = []
cp_final_list = []

for file in lsp_file_list:
    lsp_final_list = lsp_final_list + fs.glob(file)

for file in cp_file_list:
    cp_final_list = cp_final_list + fs.glob(file)

lsp_final_list = ['s3://' + item for item in lsp_final_list]
cp_final_list = ['s3://' + item for item in cp_final_list]

In [55]:
first = True
for link in lsp_final_list:
  if first:
    ds = xr.open_dataset(
    #    cached,
        link,
        engine="h5netcdf",         # or "h5netcdf"
        chunks={"time": 12},      # modest time chunks
        backend_kwargs={"storage_options": {"anon": True }},
    )
    ds = ds.sel(latitude = slice(lat_bounds[0], lat_bounds[1]), longitude = slice(lon_bounds[0], lon_bounds[1]))
    first = False
  else:
    append_ds = xr.open_dataset(
    #    cached,
        link,
        engine="h5netcdf",         # or "h5netcdf"
        chunks={"time": 12},      # modest time chunks
        backend_kwargs={"storage_options": {"anon": True }},
    )

    append_ds = append_ds.sel(latitude = slice(lat_bounds[0], lat_bounds[1]), longitude = slice(lon_bounds[0], lon_bounds[1]))

    ds = xr.concat([ds, append_ds], dim = 'forecast_initial_time')


KeyboardInterrupt: 

In [ ]:

ds = xr.open_dataset(
  #    cached,
      lsp_final_list[0],
      engine="h5netcdf",         # or "h5netcdf"
      chunks={"time": 12},      # modest time chunks
      parallel=False,           # <- important: don't delay opens
      backend_kwargs={"storage_options": {"anon": True }},
)

In [50]:
ds.sel(latitude = slice(lat_bounds[0], lat_bounds[1]), longitude = slice(lon_bounds[0], lon_bounds[1]))

<xarray.Dataset> Size: 2MB
Dimensions:                (forecast_initial_time: 30, forecast_hour: 12,
                            latitude: 41, longitude: 40)
Coordinates:
  * forecast_hour          (forecast_hour) int32 48B 1 2 3 4 5 6 7 8 9 10 11 12
  * forecast_initial_time  (forecast_initial_time) datetime64[ns] 240B 1990-0...
  * latitude               (latitude) float64 328B 52.5 52.25 ... 42.75 42.5
  * longitude              (longitude) float64 320B 232.8 233.0 ... 242.2 242.5
Data variables:
    LSP                    (forecast_initial_time, forecast_hour, latitude, longitude) float32 2MB dask.array<chunksize=(1, 12, 41, 40), meta=np.ndarray>
    utc_date               (forecast_initial_time) int32 120B dask.array<chunksize=(30,), meta=np.ndarray>
Attributes:
    DATA_SOURCE:          ECMWF: https://cds.climate.copernicus.eu, Copernicu...
    NETCDF_CONVERSION:    CISL RDA: Conversion from ECMWF GRIB1 data to netCDF4.
    NETCDF_VERSION:       4.6.1
    CONVERSION_PLATFORM:  Linux casper05 3.10.0-693.21.1.el7.x86_64 #1 SMP We...
    CONVERSION_DATE:      Wed Jun 26 08:40:59 MDT 2019
    Conventions:          CF-1.6
    NETCDF_COMPRESSION:   NCO: Precision-preserving compression to netCDF4/HD...
    history:              Wed Jun 26 08:41:05 2019: ncks -4 --ppc default=7 e...
    NCO:                  netCDF Operators version 4.7.4 (http://nco.sf.net)